# Libraries

In [1]:
import cobra as cb
import logging
logging.basicConfig(filename="log.txt"  , level=logging.INFO)
import os
import pandas as pd
import numpy as np
import math

# Set up

In [2]:
modelNames = ["ENGRO 1", "ENGRO 2"]

In [3]:
##############################################
# Create a folder if it doesn't already exist
# Parameters
# - path --> new folder path
##############################################
def createFolder(path):
    if not os.path.exists(path):
            os.mkdir(path)

# Loading models

In [4]:
##############################################
# Load the models files (.xml)
# Parameters
# - modelNames --> list of models names that must
# match the file names
# - modelFolder --> the folder containing the 
# models files
##############################################
def loadModels(modelNames, modelFolder):
    models = {}
    for modelName in modelNames:
        files = os.listdir(modelFolder)
        found = False
        for file in files:
            filename, extension = os.path.splitext(file)
            if(filename == modelName):
                found = True
                break
        if(found):
            if(extension == ".xml"):
                models[modelName] = cb.io.read_sbml_model(modelFolder + filename + extension)
            else:
                raise ImportError('Model file extension not supported')
        else:
            raise FileNotFoundError('File not found')
    return models

In [5]:
modelsDict = loadModels(modelNames, "../../models/")
modelReactionsDict = {}
for modelName in modelNames:
    listReactions = []
    for reaction in modelsDict[modelName].reactions:
        listReactions.append(reaction.id)
    modelReactionsDict[modelName] = listReactions

# Convergence analysis

In [46]:
def resultIndexAnalysis(test, value, cont):
    if(test == "geweke"):
        if(not (math.isnan(value))):
            if(abs(value) > 1.28):
                cont +=1
                return (cont)
            else:
                return (cont)
    elif(test == "raftery-lewis"):
        if(not (math.isnan(value))): # if nan --> chain too short
            if(abs(value) > 5):
                cont +=1
                return (cont)
            else:
                return (cont)
    else:
        return "ERROR"


##############################################
# It analyses the coefficients produced by convergence.r
# to create a dataframe in which each row, representing 
# a fixed number of samples, contains the fraction of times 
# in which the hypotesis of convergence must be rejected 
# for each model reaction. The last column
# of the dataframe contains the cross-reaction fraction for 
# a fixed number of samples (mean of the row fractions).
# Parameters
# - modelNames --> list of modelNames
# - modelsDict --> models dictionary
# - modelReactionsDict --> models reactions dictionary
# - tests --> diagnostics to use
# - samplesNDict --> samples dictionary for the algorithms
# - executionsPerSampleSize --> executions per sample size
# - executionsCbs3 --> executions of the CBS3 algorithm
# - thinningsDict --> thinnings dictionary
# - convergenceFolder --> folder of the convergence coefficients
# - avoidBlockedReactions --> if True, the cross-reaction mean
# does not take into account the coefficients of blocked reactions
##############################################
def convergenceAnalysis(modelNames, modelsDict, modelReactionsDict, tests, 
                        samplesNDict,executionsPerSampleSize, executionsCbs3, thinningsDict,convergenceFolder, avoidBlockedReactions):
    
    for modelName in modelNames:
        nReaction = len(modelReactionsDict[modelName])
        columns = ["Samples"]
        columns.extend(modelReactionsDict[modelName])
        columns.append("Total")
        
        if(avoidBlockedReactions):
            blockedR = cb.flux_analysis.find_blocked_reactions(modelsDict[modelName])
            nBlockedR = len(blockedR)
        
        for algorithm in algorithms:
            for thinning in thinningsDict[algorithm]:
                for test in tests:
                    if(algorithm == "cbs3"):
                        analysisFolder = os.path.join(convergenceFolder, modelName, test, algorithm + "groupedBy" + str(thinning), "analysis")
                    else:
                        analysisFolder = os.path.join(convergenceFolder, modelName, test, algorithm + "Thinning" + str(thinning), "analysis")
                    
                    createFolder(analysisFolder)
                    lenDf = 0
                    df_result = pd.DataFrame(columns =  columns)
                    contCbs3 = 0
                    for nSample in samplesNDict[algorithm]:
                        cont =  [0] * nReaction
                        
                        if(algorithm == "cbs3"):
                            executionsPerSampleSizeToUse = executionsCbs3
                        else:
                            executionsPerSampleSizeToUse = executionsPerSampleSize
                        
                        for nRun in range(0, executionsPerSampleSizeToUse):
                            if(algorithm == "cbs3"):
                                filePath =  os.path.join(convergenceFolder, modelName, test, algorithm + "groupedBy" + str(thinning), 
                                                         str(nRun) + "_" + str(0) + ".csv") 
                            else:
                                filePath =  os.path.join(convergenceFolder, modelName, test, algorithm + "Thinning" + str(thinning), 
                                                         str(nSample) + "_" + str(nRun) + ".csv") 
                            df = pd.read_csv(filePath, index_col = 1)
                            df.drop(df.filter(regex="Unname"), axis=1, inplace=True)
                            j = 0
                            for index, row in df.iterrows():
                                if(avoidBlockedReactions):
                                    if(index not in blockedR):
                                        cont[j] = resultIndexAnalysis(test, row['result'], cont[j])
                                else:
                                     cont[j] = resultIndexAnalysis(test, row['result'], cont[j])
                                    
                                j = j + 1
                                
                        for i in range(0, nReaction):
                            cont[i] = cont[i] / executionsPerSampleSizeToUse
                        if(algorithm == "cbs3"):
                            listnSample = [contCbs3]
                            contCbs3 +=1
                        else:
                            listnSample = [nSample]
                        listnSample.extend(cont)
                        summ = 0
                        for i in range (0, nReaction):
                            if(avoidBlockedReactions):
                                if(modelReactionsDict[modelName][i] not in blockedR):
                                    summ = summ + cont[i]
                            else:
                                summ = summ + cont[i]
                        if(avoidBlockedReactions):
                            listnSample.append(summ/(nReaction - nBlockedR))
                        else:
                            listnSample.append(summ/nReaction)
                        df_result.loc[lenDf] = listnSample
                        lenDf = lenDf + 1
                    df_result.set_index('Samples').to_csv(os.path.join(analysisFolder, "analysis.csv"))
    pass

In [47]:
algorithms = ["achr", "optgp", "chrr", "cbs3"]
tests = ["geweke", "raftery-lewis"]

thinningsDict = {}

avoidBlockedReactions = True

samplesNDict = {}

samplesNList = []

for i in range(1000, 30001, 1000):
    samplesNList.append(i)
    
executionsPerSampleSize = 20
executionsCbs3 = 20

samplesNDict["achr"] = samplesNList
samplesNDict["optgp"] = samplesNList
samplesNDict["chrr"] = samplesNList
samplesNDict["cbs3"] = [1000] * executionsCbs3


thinningsDict["achr"] = [1, 10, 100]
thinningsDict["optgp"] = [1, 10, 100]
thinningsDict["chrr"] = [1, 10, 100]
thinningsDict["cbs3"] = [1000]


convergenceFolder ="../../results/convergence/"

convergenceAnalysis(modelNames, modelsDict, 
               modelReactionsDict,
               tests,
               samplesNDict,
               executionsPerSampleSize,
               executionsCbs3,
               thinningsDict,
               convergenceFolder,
               avoidBlockedReactions)
